#  Introducción: Fine-tuning de un LLM para generación de ejercicios de inglés
## Objetivo del notebook

El objetivo de este notebook es realizar un fine-tuning de un LLM para adaptarlo a una tarea educativa concreta: la generación automática de ejercicios de inglés a partir de unas instrucciones dadas por el profesor.

En concreto, el modelo aprenderá a:

Recibir vocabulario específico (por ejemplo, partes de la casa, vocabulario administrativo, etc.).

Tener en cuenta un tiempo verbal concreto (por ejemplo, past simple).

Generar ejercicios estructurados y adaptados al aula (fill in the gaps, questions, short writing, etc.).
## Contexto educativo

Este trabajo está pensado para un contexto de Formación Profesional, donde el docente:

Introduce vocabulario y gramática en clase.

Necesita generar rápidamente material práctico.

Quiere mantener coherencia entre ejercicios.

El modelo resultante puede integrarse en:

- Generación de worksheets.

- Sistemas automáticos de creación de tareas.

- Flujos de trabajo con herramientas como n8n o APIs de LLM.


# Elección del modelo:

Vamos a utilizar el modelo Mistral 7b .
¿Por qué?
- Es gratuito
- Tiene buena compatibilidad con Lora y Qlora
- Según he podido investigar, es adecuado para un contexto de aprendizaje

Principales características:
- Aproximadamente 7.000 millones de parámetros.
- Es un modelo openweight, que, según vimos en clase, nos deja sus pesos abi
- Entrenado específicamente para seguir instrucciones.
- Excelente calidad de generación para su tamaño.
- Arquitectura moderna y eficiente.
- Totalmente compatible con LoRA y QLoRA.


# Instalación de librerías y comprobación de entorno

In [1]:
pip uninstall -y torch torchvision torchaudio bitsandbytes


Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Found existing installation: bitsandbytes 0.42.0
Uninstalling bitsandbytes-0.42.0:
  Successfully uninstalled bitsandbytes-0.42.0


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp312-cp312-linux_x86_64.whl (780.4 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.20.1%2Bcu121-cp312-cp312-linux_x86_64.whl (7.3 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.1%2Bcu121-cp312-cp312-linux_x86_64.whl (3.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.2.2 requires transformers<6.0.0,>=4.41.0, but you have transformers 4.38.2 which is incompatible.


In [3]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())


2.5.1+cu121
12.1
True


In [4]:
import torch

print("CUDA disponible:", torch.cuda.is_available())
print("Dispositivo:", torch.cuda.get_device_name(0))

CUDA disponible: True
Dispositivo: Tesla T4


In [5]:
%pip uninstall -y transformers tokenizers accelerate peft trl
%pip install \
  transformers==4.38.2 \
  tokenizers==0.15.2 \
  accelerate==0.27.2 \
  peft==0.9.0 \
  trl==0.7.11


Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
Found existing installation: tokenizers 0.15.2
Uninstalling tokenizers-0.15.2:
  Successfully uninstalled tokenizers-0.15.2
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
Found existing installation: peft 0.9.0
Uninstalling peft-0.9.0:
  Successfully uninstalled peft-0.9.0
Found existing installation: trl 0.7.11
Uninstalling trl-0.7.11:
  Successfully uninstalled trl-0.7.11
  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached tokenizers-0.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached accelerate-0.27.2-py3-none-any.whl.metadata (18 kB)
  Using cached peft-0.9.0-py3-none-any.whl.metadata (13 kB)
  Using cached trl-0.7.11-py3-none-any.whl.metadata (10 kB)
Using cached transformers-4.38.2-py3-none-any.whl

In [6]:
pip install bitsandbytes==0.42.0 --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 64.7 MB/s eta 0:00:00


# Carga del modelo base Mistral 7B instruct:
Para esto, usaremos además una cuantización a 4 bits. De esa manera, reduciremos el uso de memoria GPU

In [7]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

In [8]:
# Para realizar la cuantización a 4 bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [9]:
try:
    import bitsandbytes
    print("bitsandbytes is installed.")
    print(f"bitsandbytes version: {bitsandbytes.__version__}")
except ImportError:
    print("bitsandbytes is NOT installed.")

try:
    import accelerate
    print("accelerate is installed.")
    print(f"accelerate version: {accelerate.__version__}")
except ImportError:
    print("accelerate is NOT installed.")

bitsandbytes is installed.
bitsandbytes version: 0.42.0
accelerate is installed.
accelerate version: 0.27.2


In [10]:
!python -m bitsandbytes

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++ BUG REPORT INFORMATION ++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++ /usr/local CUDA PATHS +++++++++++++++++++
/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda120_nocublaslt.so
/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda121_nocublaslt.so
/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda114.so
/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so
/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so
/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda115.so
/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda111.so
/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda120.so
/u

In [11]:
# Carga del tokenizador

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=False
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
# Carga del modelo cuantizado
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
# Comprobación del dispositivo en el que está trabajando el modelo
print("Modelo cargado en:", model.device)


Modelo cargado en: cuda:0


In [14]:
#prueba del modelo:
prompt = "Create a short English exercise using the past simple tense."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Create a short English exercise using the past simple tense.

Title: A Day in the Life of a Student

Instructions:

Read the text below and answer the questions that follow.

Text:

Yesterday, I went to the university. I took the bus at 8:30 a.m. and arrived at the campus at 9:15. My first class started at 9:30 and it was English. We learned new vocabulary words and practiced speaking in groups. After


Una vez que tenemos esto hecho, vamos a dar el siguiente paso, que será, cargar el dataset de ejemplos de tareas en un json. Este dataset es sintético, ha sido extraído de chatgpt tras haberle dado algunas directrices del tipo de tareas que debería crear.

In [15]:
# Importamos la función para cargar datasets desde Hugging Face
from datasets import load_dataset

# Cargamos el dataset desde un archivo JSONL.
# Cada línea del archivo corresponde a un ejemplo de entrenamiento.
dataset = load_dataset(
    "json",
    data_files="dataset.jsonl",
    split="train"
)

# Mostramos un resumen del dataset para verificar que se ha cargado correctamente
dataset


Dataset({
    features: ['messages'],
    num_rows: 60
})

In [16]:
#comprobamos el dataset
dataset[0]


{'messages': [{'role': 'system',
   'content': 'You are an English teacher for vocational training students. You create vocabulary exercises adapted to A2 level.'},
  {'role': 'user',
   'content': 'Create a vocabulary exercise about office objects.'},
  {'role': 'assistant',
   'content': 'Exercise 1: Match the words with the correct object.\n\n1. Printer\n2. Desk\n3. Chair\n4. Computer\n\nA. Used to sit\nB. Used to print documents\nC. Used to work with files\nD. Used to place papers'}]}

In [17]:
# Mostramos las columnas (features) del dataset para comprobar su estructura
print(dataset.features)

# Mostramos el número total de ejemplos de entrenamiento
print(dataset.num_rows)

{'messages': List({'role': Value('string'), 'content': Value('string')})}
60


In [18]:
def format_prompt(example):
    # Convertimos la conversación (messages) en un único texto de entrenamiento
    prompt = ""

    for message in example["messages"]:
        role = message["role"].upper()
        content = message["content"]
        prompt += f"[{role}]\n{content}\n\n"

    return {"text": prompt.strip()}

In [19]:
dataset = dataset.map(format_prompt)
print(dataset[0]["text"])

[SYSTEM]
You are an English teacher for vocational training students. You create vocabulary exercises adapted to A2 level.

[USER]
Create a vocabulary exercise about office objects.

[ASSISTANT]
Exercise 1: Match the words with the correct object.

1. Printer
2. Desk
3. Chair
4. Computer

A. Used to sit
B. Used to print documents
C. Used to work with files
D. Used to place papers


In [20]:
#Tokenización del dataset
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        max_length=512,
        padding=False
    )
# Añadimos labels al dataset para un correcto entrenamiento
    tokens["labels"] = tokens["input_ids"].copy()

    return tokens

tokenized_dataset = dataset.map(
    tokenize,
    remove_columns=dataset.column_names
)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Una vez hemos preparado el modelo y el dataset para nuestra tarea, podemos empezar a entrenar el proceso en 4- bit

In [21]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
model = prepare_model_for_kbit_training(model)

In [22]:
#Configuración de LoRA
#Definimos los parámetros que controlan qué partes del modelo se entrenan

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05, #esta capa evita el sobreajuste durante el entrenamiento
    bias="none",
    task_type="CAUSAL_LM"
)

In [23]:
# aplicamos lora al modelo:
model = get_peft_model(model, lora_config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer):

In [24]:
#Comprobamos el número de parámetros a entrenar.
model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 7,283,675,136 || trainable%: 0.5758499550960753


In [25]:
from transformers import Trainer, TrainingArguments
# Definimos los parámetros del entrenamiento
training_args = TrainingArguments(
    output_dir="./results",            # Carpeta donde se guardan los resultados
    num_train_epochs=3,                # Número de épocas de entrenamiento
    per_device_train_batch_size=1,     # Batch size por GPU
    gradient_accumulation_steps=4,     # Simula un batch mayor acumulando gradientes
    learning_rate=2e-4,                # Learning rate típico para LoRA
    fp16=True,                         # Entrenamiento en media precisión
    logging_steps=10,                  # Frecuencia de logging
    save_strategy="epoch",             # Guardar el modelo al final de cada época
    report_to="none"                   # Desactivamos servicios externos de logging
)


In [26]:
# Inicializamos el entrenador para fine-tuning supervisado
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [27]:
# Comenzamos el proceso de entrenamiento
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.721700
20,0.609700
30,0.369600
40,0.174600


Checkpoint destination directory ./results/checkpoint-15 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Checkpoint destination directory ./results/checkpoint-30 already exists and is non-empty. Saving wil

TrainOutput(global_step=45, training_loss=0.6585872809092204, metrics={'train_runtime': 172.9147, 'train_samples_per_second': 1.041, 'train_steps_per_second': 0.26, 'total_flos': 642961801101312.0, 'train_loss': 0.6585872809092204, 'epoch': 3.0})

Con nuestro modelo ya entrenado, vamos a hacer pruebas de éste para ver cómo funciona en distintos ejercicios. Pese a que aquí incluyo una sola versión, este prompt ha pasado por varias etapas de refinamiento hasta encontrar un resultado satifactorio

Vamos a probar los siguientes ejercicios:
-Word formation
-Reading comprehension + preguntas
-Grammar (Fill in the gaps)

In [52]:
# Ejercicio word- formation
# Lista de vocabulario a probar: describe, king, science, apology, protect, warm, art, die, true, empire, locate, select, fascinate, luck, poem, mystery, succeed, destroy, honest, high, consider, hope, meaning, appear, nation

# =========================
# INPUT DEL USUARIO
# =========================

# El usuario introduce las palabras separadas por comas
vocabulary_input = input("Enter a list of words separated by commas: ")
templates = [
    "She gave a clear ___ of the problem.",
    "He works in the field of ___.",
    "The documents were under company ___.",
    "His sudden ___ shocked everyone.",
    "We discussed the real ___ of the text.",
    "She became a famous ___.",
    "The final ___ was announced yesterday.",
    "With a bit of ___, everything is possible."
]
# Convertimos el texto en una lista de palabras limpia
vocabulary = [word.strip() for word in vocabulary_input.split(",")]
# =========================
# PROMPT COMPLETO
# =========================
prompt = f"""[SYSTEM]
You are an English teacher creating B1 word formation exercises.

[USER]
Use ONLY the sentence templates below to create a word formation exercise similar to this one.

Sentence templates:
1. She gave a clear ______ of the problem. (DESCRIBE)
Answer: description

2. He works in the field of ______. (SCIENCE)
Answer: science

3. The documents were under company ______. (PROTECT)
Answer: protection

4. His sudden ______ shocked everyone. (APPEAR)
Answer: appearance

5. We discussed the real ______ of the text. (MEANING)
Answer: meaning



For each sentence:
-  Create the sentence from scratch
- Give a prompt (a word in brackets) that serves the student as a base to create the answer to the exercise
- Make sure the exercise requires the student to write a word with preffixes or suffixes
- Add the base word in CAPITAL LETTERS in brackets at the end of each sentence
- Check a second time that all the sentences imply that the base word or prompt must be changed by the student




"""
# =========================
# INFERENCIA CONTROLADA
# =========================
# Tokenizamos el prompt
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

prompt_length = inputs["input_ids"].shape[1]

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=250,
        do_sample=False,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id
    )

# Nos quedamos SOLO con los nuevos tokens (la respuesta)
generated_tokens = outputs[0][prompt_length:]

result = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print(result)

Enter a list of words separated by commas: describe, king, science, apology, protect, warm, art, die, true, empire, locate, select, fascinate, luck, poem, mystery, succeed, destroy, honest, high, consider, hope, meaning, appear, nation


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[ASSISTANT]
Exercise:

1. She gave a clear ______ of the problem. (DESCRIBE)
2. He works in the field of ______. (SCIENCE)
3. The documents were under company ______. (PROTECT)
4. His sudden ______ shocked everyone. (APPEAR)
5. We discussed the real ______ of the text. (MEANING)

Answers:
1. description
2. science
3. protection
4. appearance
5. meaning

Prompts:
1. describe
2. scientific
3. protect
4. appear
5. mean

Explanation:
The exercise requires students to change words by adding prefixes or suffixes. The prompts give the base form for the answers. For example, the prompt "describe" suggests the answer "description". All sentences imply that the base word must be changed by the student.


# Evaluación ejercicio Word formation:
Tras refinar el prompt, he comprobado que estos ejercicios no son fáciles de conseguir de manera útil. En este caso, no tiene sentido informar al prompt del tiempo verbal en el que deberían contextualizar el ejercicio, dado que el LLM entiende que debe hacer un ejercicio basado en tiempos verbales y gramática.

Exercise:

1. She gave a clear ______ of the problem. (DESCRIBE)
2. He works in the field of ______. (SCIENCE)
3. The documents were under company ______. (PROTECT)
4. His sudden ______ shocked everyone. (APPEAR)
5. We discussed the real ______ of the text. (MEANING)

Answers:
1. description
2. science
3. protection
4. appearance
5. meaning

Prompts:
1. describe
2. scientific
3. protect
4. appear
5. mean

Como vemos, en este caso ha creado distntas palabras, todas ellas para ser cambiadas a sustantivo, o nominalizadas.
Este resultado denota que el modelo encuentra dificultades en la variedad de transformaciones (no crea adverbios, adjetivos, verbos...)

Si bien estos ejercicios son aceptables como un primer acercamiento, no nos da una variedad satisfactoria como para pensar que este caso de uso ha sido útil

In [47]:
#grammar: past simple
#vocabulary: hiking, camping, rock climbing, cycling, kayaking, canoeing, surfing, paddle boarding, skiing, snowboarding, trail running, mountain biking, nature walk, outdoor workout, team sports, adventure sports, physical activity, fresh air, natural environment, open spaces, equipment, safety rules, protective gear, helmet, comfortable clothing, weather conditions, sunny day, rainy weather, cold temperatures, warm temperatures, risk, challenge, endurance, strength, balance, coordination, teamwork, motivation, mental health, physical health, stress reduction, well-being, healthy lifestyle, free time, leisure activities, outdoor experience
# Ejercicio: Reading comprehension
# =========================
# INPUT DEL USUARIO
# =========================

grammar = input("Include the grammar focus (e.g. past simple, modal verbs, conditionals): ")

vocabulary_input = input(
    "Enter a list of vocabulary words separated by commas: "
)

# Convertimos el vocabulario en una lista limpia
vocabulary = [word.strip() for word in vocabulary_input.split(",")]

# =========================
# PROMPT COMPLETO
# =========================

prompt = f"""[SYSTEM]
You are an English teacher for vocational training students. You create reading comprehension activities adapted to B1 level.

[USER]
Create a reading comprehension activity.

The activity must focus on the following grammar topic:
{grammar}

Try to naturally include the following vocabulary in the text:
{", ".join(vocabulary)}

PART 1 – READING TEXT
- Write a short reading text of about 180–220 words.
- The text must be appropriate for B1 level.
- Use clear language and realistic situations related to everyday life or vocational contexts.
- Avoid overly technical vocabulary.
- Do not include titles or headings.

PART 2 – COMPREHENSION QUESTIONS
- Write EXACTLY 5 comprehension questions based on the text.
- The questions must test understanding of ideas, reasons, consequences, or implicit information.
- Do NOT copy phrases or sentences directly from the text.
- Do NOT ask questions that can be answered by matching the same words from the text.
- Paraphrase ideas when forming the questions.
- Avoid very obvious factual questions (names, dates, places).
- Use different question types (why, how, what can be inferred, what is the main idea).
- Do NOT include the answers.
- Stop writing immediately after the 5th question.

Write the reading text first, then the questions.

[ASSISTANT]
"""

# =========================
# INFERENCIA CONTROLADA
# =========================
# Tokenizamos el prompt
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

prompt_length = inputs["input_ids"].shape[1]

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=250,
        do_sample=False,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id
    )

# Nos quedamos SOLO con los nuevos tokens (la respuesta)
generated_tokens = outputs[0][prompt_length:]

result = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print(result)


Include the grammar focus (e.g. past simple, modal verbs, conditionals): conditionals
Enter a list of vocabulary words separated by commas: hiking, camping, rock climbing, cycling, kayaking, canoeing, surfing, paddle boarding, skiing, snowboarding, trail running, mountain biking, nature walk, outdoor workout, team sports, adventure sports, physical activity, fresh air, natural environment, open spaces, equipment, safety rules, protective gear, helmet, comfortable clothing, weather conditions, sunny day, rainy weather, cold temperatures, warm temperatures, risk, challenge, endurance, strength, balance, coordination, teamwork, motivation, mental health, physical health, stress reduction, well-being, healthy lifestyle, free time, leisure activities, outdoor experience


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Reading Text:
Yesterday, Mark went hiking with his friends. They prepared their backpacks and checked the safety rules before starting the trail. Mark enjoyed the beautiful views and the fresh air. After the hike, they had lunch at a campsite and did some rock climbing. In the afternoon, they went cycling and kayaking. Mark felt energized and motivated. He was grateful for the opportunity to spend time in nature and disconnect from work.

Comprehension Questions:
1. Why did Mark go hiking?
2. What activities did he do with his friends?
3. How did he feel after the day?
4. What benefits did he get from spending time outdoors?
5. What did he appreciate most about the day?


# Evaluación ejercicio Reading Comprehension
## Conclusiones sobre el ejercicio de reading comprehension con enfoque gramatical (conditionals)

El ejercicio de comprensión lectora generado a partir de un texto contextualizado en actividades al aire libre presenta un nivel lingüístico adecuado para alumnado de B1. El texto es claro, coherente y utiliza un léxico relacionado con actividades físicas y tiempo libre, lo que favorece la motivación y la comprensión global del contenido.

En cuanto a la integración de la **gramática (conditionals)**, se observa que el texto proporciona un contexto narrativo válido, pero la presencia de la estructura gramatical no es explícita ni se explota directamente en las preguntas de comprensión. Esto es coherente con un enfoque de lectura cuyo objetivo principal es la comprensión del significado, y no la evaluación directa de la gramática.

Respecto a las **preguntas de comprensión**, se ha detectado que la mayoría son de carácter literal. Varias de ellas pueden responderse mediante la localización directa de información en el texto, sin necesidad de realizar inferencias o reformular ideas. Esto limita el desarrollo de estrategias de comprensión más profundas, como la deducción de consecuencias, la interpretación de intenciones o la reflexión personal a partir del contenido leído.

Para mejorar el valor didáctico del ejercicio, especialmente en relación con el enfoque gramatical en conditionals, sería recomendable:
- formular preguntas que inviten al alumnado a imaginar situaciones alternativas o consecuencias hipotéticas relacionadas con el texto,
- promover inferencias basadas en el contenido (por ejemplo, qué ocurriría si las condiciones fueran diferentes),
- evitar preguntas que reproduzcan el mismo vocabulario o estructura que aparece de forma explícita en el texto.

En conclusión, el ejercicio es válido como práctica de comprensión lectora general, pero puede optimizarse mediante un diseño de preguntas más inferencial y una explotación más clara del enfoque gramatical propuesto. Esto permitiría un mejor equilibrio entre comprensión lectora y reflexión lingüística, alineándose con los objetivos comunicativos del nivel B1.

ChatGPT nos muestra la siguiente propuesta de preguntas utilizando el mismo texto: 

1. What does the text suggest about Mark’s routine before this trip?
2. Why do you think Mark enjoyed the day more than a normal workday?
3. How did the different activities contribute to Mark’s mood?
4. What can be inferred about the importance of nature for Mark?
5. In what way did the day help Mark personally?

In [51]:
import torch

# =========================
# INPUT DEL USUARIO
# =========================

grammar_1 = input("Enter the first grammar tense (e.g. present simple): ")
grammar_2 = input(
    "Enter a second grammar tense for contrast (e.g. past continuous): "
)

# =========================
# PROMPT COMPLETO
# =========================

prompt = f"""[SYSTEM]
You are an English teacher for vocational training students. You create grammar exercises adapted to B1 level.

[USER]
Create TWO grammar exercises (fill in the gaps).

EXERCISE 1 – Single grammar focus
- Focus on the following grammar topic: {grammar_1}
- Write EXACTLY 5 sentences.
- Each sentence must contain ONE gap.
- Students must complete the gap using the correct grammatical form.
- Do NOT include the answers.

EXERCISE 2 – Grammar contrast
- Focus on the contrast between these two grammar topics:
  {grammar_1} vs {grammar_2}
- Write EXACTLY 5 sentences.
- Each sentence must contain ONE gap.
- Each sentence must clearly require choosing between the two grammar forms.
- Use realistic and clear contexts.
- Do NOT include the answers.

Write Exercise 1 first, then Exercise 2.
Stop writing after the last sentence of Exercise 2.

[ASSISTANT]
"""

# =========================
# INFERENCIA CONTROLADA
# =========================
# Tokenizamos el prompt
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

prompt_length = inputs["input_ids"].shape[1]

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=250,
        do_sample=False,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id
    )

# Nos quedamos SOLO con los nuevos tokens (la respuesta)
generated_tokens = outputs[0][prompt_length:]

result = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print(result)

Enter the first grammar topic (e.g. present simple): past simple
Enter a second grammar topic for contrast (e.g. past continuous): past continuous


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Exercise 1:
1. Yesterday, I ______ (meet) a client.
2. She ______ (work) late yesterday.
3. We ______ (finish) the task at noon.
4. He ______ (have) a meeting at ten.
5. They ______ (start) the project last week.

Exercise 2:
1. Yesterday, I ______ (meet) a client. (past simple)
2. She ______ (work) late yesterday. (past simple)
3. We ______ (finish) the task at noon. (past simple)
4. He ______ (have) a meeting at ten. (past simple)
5. They ______ (start) the project last week. (past simple)
6. Yesterday, she ______ (write) the report. (past continuous)
7. Last night, he ______ (check) the emails. (past continuous)
8. This morning, they ______ (prepare) the documents. (past continuous)


# Valoración ejercicios fill in the gaps (grammar)
Los ejercicios de gramática generados mediante el modelo muestran resultados desiguales en función del tipo de tarea planteada y del nivel de control aplicado al prompt.

En los ejercicios de **práctica de un solo tiempo verbal** (por ejemplo, *past simple*), el modelo ofrece resultados satisfactorios. Las frases muestran expresiones de tiempo claras, un único hueco por oración y un nivel adecuado para alumnado de B1. Este tipo de ejercicio es sencillo de generar automáticamente, ya que no requiere toma de decisiones complejas por parte del estudiante.

Sin embargo, en los ejercicios de **contraste entre dos tiempos verbales** (como *past simple* vs *past continuous*), se ha observado que el modelo tiende inicialmente a producir frases que no evalúan el contraste real, limitándose a la conjugación de un único tiempo o incluso indicando explícitamente qué forma debe usarse. Esto reduce el valor pedagógico del ejercicio y elimina el componente de reflexión gramatical.

Para que los ejercicios de contraste sean válidos desde un punto de vista didáctico, es necesario que:
- cada frase incluya un contexto que obligue a elegir entre los dos tiempos verbales,
- se presenten dos acciones relacionadas (acción en progreso e interrupción, acciones simultáneas, etc.),
- no se indique explícitamente qué tiempo verbal debe emplearse.

Tras ajustar el diseño del ejercicio y el prompt, los resultados mejoran significativamente, generando frases más cercanas a las que aparecen en pruebas de evaluación reales de nivel B1.

En conclusión, el uso de modelos de lenguaje para la generación de ejercicios gramaticales es especialmente eficaz cuando:
- el tipo de ejercicio está bien definido,
- el grado de libertad del modelo es limitado,
- y se controla cuidadosamente el formato y la intención evaluativa.


## Conclusiones



Los resultados de ejercicios que hemos obtenido con el modelo Mistral 7b instruct fine-tuneado con un dataset artificial han sido, si bien interesantes, aún no satisfactorios para darle un uso profesional del cual profesores y alumnos puedan sacar partido para practicar, generar modelos para exámenes, etc. 

Uno de los principales problemas detectados es la dificultad del modelo para interpretar correctamente finalidad de la tarea cuando la tarea exige un control muy preciso del tipo de respuesta esperada. En estos casos, el modelo tiende a priorizar la corrección gramatical general frente a la utilidad específica del ejercicio, creando un ejercicio inacabado, incompleto, o que requiere muy poco esfuerzo por parte del alumno

Aun así, los resultados pueden considerarse prometedores. El experimento demuestra que, mediante un prompting habilidoso y un proceso de entrenamiento adecuado, los modelos de lenguaje pueden generar materiales educativos funcionales y coherentes. El uso de Mistral 7B Instruct resulta especialmente interesante por tratarse de un modelo gratuito, lo que refuerza su atractivo en entornos educativos con recursos limitados, a pesar de las limitaciones observadas.

De cara a trabajos futuros, se plantean varias líneas de mejora y exploración:
- Probar tareas menos dependientes de estructuras cerradas, como actividades de *speaking* o *writing*, donde el modelo puede explotar mejor su capacidad generativa.
- Experimentar con modelos de mayor tamaño y número de parámetros, manteniendo los mismos criterios de fine-tuning, para analizar si se obtiene una mejora significativa en la precisión y control de las respuestas.
- Comparar el rendimiento del modelo entrenado con otros modelos más avanzados, como Gemini, con el fin de evaluar hasta qué punto el tamaño y la arquitectura influyen en la generación de materiales didácticos de calidad.

En conclusión, aunque el modelo utilizado aún no ofrece resultados plenamente satisfactorios para todas las tipologías de ejercicios, el trabajo realizado evidencia el potencial de los modelos de lenguaje como herramientas de apoyo docente, siempre que su uso vaya acompañado de un diseño pedagógico sólido y de una supervisión humana constante.
